A Jupyter Notebook script for scraping data from the NCBI website
    
## Install

In [26]:
!pip install  BeautifulSoup

You are using pip version 18.1, however version 20.3.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [27]:
!pip install  pandas

You are using pip version 18.1, however version 20.3.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [28]:
!pip install  tqdm

You are using pip version 18.1, however version 20.3.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [29]:
!pip install requests

You are using pip version 18.1, however version 20.3.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Import

In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm


In [31]:

tqdm.pandas()
def find_submited_by(acc_arg):
    Answer = dict()
    ACC  = str(acc_arg)
    URL = f'https://www.ncbi.nlm.nih.gov/sra/?term={ACC}'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    div_results = soup.find_all('div',class_='sra-full-data')
    for num, i in enumerate(div_results):
        string=i.getText()
        string = string.replace(u'\xa0', u' ')
        if "Submitted by:" in string:
            Answer['ACC'] = ACC
            Answer['Submitted_by'] = string.split(':')[1]
            return string.split(':')[1]
def find_ID(acc_arg):
    Answer = dict()
    ACC  = str(acc_arg)
    URL = f'https://www.ncbi.nlm.nih.gov/sra/?term={ACC}'
    page = requests.get(URL)    
    soup1 = BeautifulSoup(page.content, 'html.parser') 
    div_results2 = soup1.find_all('div',class_='resc')
    for num, i in enumerate(div_results2):
        string=i.getText()
        string = string.replace(u'\xa0', u' ')
        if "ID:" in string:
            Answer['ACC'] = ACC
            Answer['ID'] = string.split(':')[1]
            return string.split(':')[1]
def find_submission_biosample_field(ID_arg):
    Answer = dict()
    ID     = str(ID_arg)
    URL   = f'https://www.ncbi.nlm.nih.gov/biosample?LinkName=sra_biosample&from_uid={ID}'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')  
    div_results = soup.find_all('div',class_='docsum')
    for num, i in enumerate(div_results):
        string=i.getText()
        string = string.replace(u'\xa0', u'')
        if "Submission" in string:
            return string.split('Submission')[1].split(';')[0]

def find_submission_bioproject_field(ID_arg):
    ID = str(ID_arg)
    URL = f'https://www.ncbi.nlm.nih.gov/bioproject?LinkName=sra_bioproject&from_uid={ID}'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')  
    div_results = soup.find_all('table', id='CombinedTable')
    if div_results:
        submission_info = div_results[0].find('td', text='Submission')
        if submission_info is not None:
            return submission_info.find_next_sibling('td').find('b').get_text()
    return None

In [32]:
df = pd.read_csv('./data/input/sample_data.csv')
df['Submitted_by'] = df['Acc'].progress_apply(find_submited_by)
df['ID'] = df['Acc'].progress_apply(find_ID)
df['Biosample_Submission'] = df['ID'].progress_apply(find_submission_biosample_field)
df['Bioproject_Submission'] = df['ID'].progress_apply(find_submission_bioproject_field)

100%|██████████| 37/37 [00:35<00:00,  1.03it/s]


Reorder the columns 

In [33]:
df = df[['Acc', 'ID', 'Center_Names', 'Submitted_by', 'Biosample_Submission','Bioproject_Submission']]


In [34]:
df=df.fillna('')
df.head(50)

,Acc,ID,Center_Names,Submitted_by,Biosample_Submission,Bioproject_Submission
0,SRR8606903,7314038,AUSTRALIAN INSTITUTE OF MARINCE SCIENCES,Australian Institute of Marince sciences,"Australian Institute of Marince sciences, Patr...",
1,SRR11788653,10848896,CLARK UNIVERSITY,Clark University,"Clark University, Nathan Ahlgren",Clark University
2,SRR9158324,7959983,CALIFORNIA STATE UNIVERSITY FULLERTON,California State University Fullerton,"California State University Fullerton, Maria R...",California State University Fullerton
3,SRR11881309,10964535,"CALIFORNIA STATE UNIVERSITY, FULLERTON","California State University, Fullerton","California State University, Fullerton, Joshua...","California State University, Fullerton"
4,SRR10293930,9209919,"CALIFORNIA STATE UNIVERSITY, LOS ANGELES","California State University, Los Angeles","California State University, Los Angeles, Kirs...","California State University, Los Angeles"
5,SRR8665997,7378698,"CALIFORNIA STATE UNIVERSITY, NORTHRIDGE","California State University, Northridge","California State University, Northridge, Kadir...","California State University, Northridge"
6,SRR10018586,8900739,CFSAN,FDA Center for Food Safety and Applied Nutrit...,CFSAN,FDA/CFSAN
7,SRR8606903,7314038,AUSTRALIAN INSTITUTE OF MARINCE SCIENCES,Australian Institute of Marince sciences,"Australian Institute of Marince sciences, Patr...",
8,SRR10406092,9345478,AUSTRALIAN INSTITUTE OF MARINE SCIENCE,Australian Institute of Marine Science,"Australian Institute of Marine Science, Heidi ...",Australian Institute of Marine Science
9,SRR10566897,9518689,AUSTRALIAN INSTITUTE OF TROPICAL HEALTH AND ME...,Australian Institute of Tropical Health and M...,Australian Institute of Tropical Health and Me...,Los Alamos National Laboratory


In [35]:
df.to_csv('./data/output/sample_data_Scrape_Result.csv', encoding='utf-8')
